In [1]:
# import libraries
import os
import time
import json, requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from dotenv import load_dotenv
load_dotenv()

client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")
version = '20180604'
limit = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore'

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium 

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="capstone project app")

# get html from url using lxml parser and return only data within the wikitable sortable class
resp = requests.get('https://www.blumenau.sc.gov.br/secretarias/secretaria-de-desenvolvimento-urbano/pagina/historia-sobre-municipio/divisa-administrativa-bairros').text
soup = BeautifulSoup(resp, 'lxml')
data = soup.find('div',{'id':'ultimas'})

print('Your credentails:')
print('CLIENT_ID: ' + client_id)
print('CLIENT_SECRET: ' + client_secret)

Your credentails:
CLIENT_ID: XXJVQIWYIHG5SOQMR4HDRJAZVWUKZ3EQ0QRF2VLJALDIV34C
CLIENT_SECRET: VFIELNNCKOIEQR4GUCMHJ0PODS5ZAPBYK0ZABKKGZGYNJ33J


In [2]:
# iterate through rows and save as multideminsional array
bairros = []
for row in data.findAll('li')[1:]:
    cells = row.find_all(['span'])
   
    try:
        
        if(cells[0].text== 'Bairro Vorstardt'):
            bairro = 'Bairro Vorstadt'
        else:
            bairro = cells[0].text
        
        
    except IndexError:
        continue

    bairros.append(bairro.rstrip())

print(bairros)

['Bairro Água Verde', 'Bairro Badenfurt', 'Bairro Boa Vista', 'Bairro Bom Retiro', 'Bairro Centro', 'Bairro Da Glória', 'Bairro Do Salto', 'Bairro Escola Agrícola', 'Bairro Fidélis', 'Bairro Fortaleza', 'Bairro Fortaleza Alta', 'Bairro Garcia', 'Bairro Itoupava Central', 'Bairro Itoupava Norte', 'Bairro Itoupava Seca', 'Bairro Itoupavazinha', 'Bairro Jardim Blumenau', 'Bairro Nova Esperança', 'Bairro Passo Manso', 'Bairro Ponta Aguda', 'Bairro Progresso', 'Bairro Ribeirão Fresco', 'Bairro Salto do Norte', 'Bairro Salto Weissbach', 'Bairro Testo Salto', 'Bairro Tribess', 'Bairro Valparaíso', 'Bairro Velha', 'Bairro Velha Central', 'Bairro Velha Grande', 'Bairro Victor Konder', 'Bairro Vila Formosa', 'Bairro Vila Itoupava', 'Bairro Vila Nova', 'Bairro Vorstadt']


In [3]:
# create dataFrame from data and assign column names
df = pd.DataFrame(bairros, columns=['Bairros'])
df['Bairros'] = df['Bairros'].map(lambda x: str(x)[7:])

df

,Bairros
0,Água Verde
1,Badenfurt
2,Boa Vista
3,Bom Retiro
4,Centro
5,Da Glória
6,Do Salto
7,Escola Agrícola
8,Fidélis
9,Fortaleza


In [6]:
# def locate(x):
#     try:
#         location = geolocator.geocode('blumenau {}'.format(x))
#         print(x, location.latitude, location.longitude)
#     except:
#         time.sleep(2)
#         location = geolocator.geocode('blumenau {}'.format(x))
#         print(x, location.latitude, location.longitude)
#     time.sleep(2)
#     return location.latitude, location.longitude

# df["Latitude"], df["Longitude"] = zip(*df["Bairros"].map(locate))

# in fact we could just use the coords file, but in case we add addition columns in future df I will leave this way
latlong = pd.read_csv('coords.csv')
df = pd.merge(df, latlong, on='Bairros')

df.head()

,Bairros,Latitude,Longitude
0,Água Verde,-26.910743,-49.107369
1,Badenfurt,-26.883060,-49.135753
2,Boa Vista,-26.901357,-49.066842
3,Bom Retiro,-26.925561,-49.071635
4,Centro,-26.919902,-49.065934
